# Extracción de Datos de Interés

## Importación de librerias

In [19]:
#Cargamos las librerias 
import numpy as np
import pandas as pd
import json
import re
from tqdm.notebook import tqdm
import polars as pl
tqdm.pandas()

## Carga de datos


In [11]:
#pagos_df = pd.read_excel('..\\Tests_files\\ComplementoPagosHardTest.xlsx')
pagos_df = pd.read_csv('E:\Desarrollos\extract_json_vertica_python\\app\\temp\FROM_Vertica_Complemento_pago.csv')
pagos_df.head()



,tfdUUID,ComplementoPagos
0,04D4F5FA-B3AB-4039-A21A-59BD02E28919,"{\n\t""pago10:Pagos"": {\n\t\t""_Version"": ""1.0"",..."
1,04D72CEF-E572-4952-8EFF-EFAEC80C186F,"{\n\t""pago10:Pagos"": {\n\t\t""_Version"": ""1.0"",..."
2,04D7741E-46DD-48EF-8268-923C64031CE4,"{\n\t""pago10:Pagos"": {\n\t\t""_Version"": ""1.0"",..."
3,04DA0999-D7B5-4E36-AA4F-4F36112025DF,"{\n\t""pago10:Pagos"": {\n\t\t""_Version"": ""1.0"",..."
4,04DA7BFE-E75A-416F-B285-012447570552,"{\n\t""pago10:Pagos"": {\n\t\t""_Version"": ""1.0"",..."


In [12]:
complementoPagos_Json = json.loads(pagos_df['ComplementoPagos'][0])
complemento_Text = str(complementoPagos_Json)
keys_to_find = ["_IdDocumento", "pago20:ImpuestosDR","pago10:ImpuestosDR"]
keys_to_extract = keys_to_find.copy() 
def find_dicts_with_keys(d, keys, test_key):
    found_dicts = []
    if isinstance(d, dict):
        if any(key in d for key in keys):
            found_dicts.append(d)
        for value in d.values():
            if isinstance(value, (dict, list)):
                found_dicts.extend(find_dicts_with_keys(value, keys,test_key))
    elif isinstance(d, list):
        for item in d:
            found_dicts.extend(find_dicts_with_keys(item, keys,test_key))
            
    found_dicts = [d for d in found_dicts if test_key in d ]
    return found_dicts




def extract_keys_from_dicts(dicts, keys_to_extract):
    extracted_dicts = []
    for d in dicts:
        extracted_dict = {}
        for key in keys_to_extract:
            if key in d:
                extracted_dict[key] = d[key]
        extracted_dicts.append(extracted_dict)
    return extracted_dicts

# Extraer las llaves específicas de los diccionarios encontrados


def add_tfduuid_id(tfduuid, extracted_dicts):
    dicts_with_id = []
    for ind, dict_ in enumerate(extracted_dicts):
        if isinstance(dict_, dict):
            dicts_with_id.append({"_IdDocumento": tfduuid,  **dict_})  #"id": ind+1,
    return dicts_with_id
result_dicts = find_dicts_with_keys(complementoPagos_Json, keys_to_find,'_IdDocumento')
extracted_dicts = extract_keys_from_dicts(result_dicts, keys_to_extract)
add_tfduuid_id("123234", extracted_dicts)
result_dicts




[{'_Folio': 'D0000115064',
  '_IdDocumento': 'D8378007-32BC-4175-9801-3F5A232A43F2',
  '_ImpPagado': '36549.00',
  '_ImpSaldoAnt': '36549.00',
  '_ImpSaldoInsoluto': '0',
  '_MetodoDePagoDR': 'PPD',
  '_MonedaDR': 'MXN',
  '_NumParcialidad': '1',
  '_Serie': 'CFDI'},
 {'_Folio': 'D0000115064',
  '_IdDocumento': '9C13BBB2-EE88-40A4-868D-CCA7829AC488',
  '_ImpPagado': '30089.00',
  '_ImpSaldoAnt': '30089.00',
  '_ImpSaldoInsoluto': '0',
  '_MetodoDePagoDR': 'PPD',
  '_MonedaDR': 'MXN',
  '_NumParcialidad': '1',
  '_Serie': 'CFDI'},
 {'_Folio': 'D0000115064',
  '_IdDocumento': 'A3CD0EDF-E707-4732-89F4-2925DDCDF537',
  '_ImpPagado': '19115.00',
  '_ImpSaldoAnt': '19115.00',
  '_ImpSaldoInsoluto': '0',
  '_MetodoDePagoDR': 'PPD',
  '_MonedaDR': 'MXN',
  '_NumParcialidad': '1',
  '_Serie': 'CFDI'},
 {'_Folio': 'D0000115064',
  '_IdDocumento': '84FEED47-E133-43FE-BF18-3E36AD16A77E',
  '_ImpPagado': '19115.00',
  '_ImpSaldoAnt': '19115.00',
  '_ImpSaldoInsoluto': '0',
  '_MetodoDePagoDR': 'PPD

In [13]:
def functionOne_ImpuestosDRCF(row):
    keys_to_find = ['_IdDocumento',"pago20:ImpuestosDR","pago10:ImpuestosDR" ]
    keys_to_extract = keys_to_find.copy() 
    uuid = "123456789"
    	
    try:
        uuid = row['tfdUUID']
        traslados = row['ComplementoPagos']
        traslados_Text = str(traslados)
        cadena_json = traslados_Text.replace("'", '"')  # Reemplazar comillas simples por comillas dobles
        traslados_Json = json.loads(cadena_json)
        keys_to_find = ['_IdDocumento',"pago20:ImpuestosDR","pago10:ImpuestosDR" ]
        keys_to_extract = keys_to_find.copy()
        result_dicts = find_dicts_with_keys(traslados_Json, keys_to_find,'_IdDocumento')
        extracted_dicts = extract_keys_from_dicts(result_dicts, keys_to_extract)
        result = add_tfduuid_id(uuid, extracted_dicts)
    except Exception as e:
        print(e)
        default_values = {'pago20:ImpuestosDR': {'pago20:TrasladosDR': {'pago20:TrasladoDR': {'_BaseDR': '00',
     '_ImporteDR': '000',
     '_ImpuestoDR': '000',
     '_TasaOCuotaDR': '0.0000',
     '_TipoFactorDR': '0'}}}}
        result = [{'_IdDocumento': uuid, **default_values}]#'id': 00, 
    return result

In [15]:
def functionTwo_ImpuestosDRCF(df):
    results = df.apply(functionOne_ImpuestosDRCF, axis=1).tolist()
    flat_list = [item for sublist in results for item in sublist]
    return flat_list
x = functionTwo_ImpuestosDRCF(pagos_df)
x

[{'_IdDocumento': 'D8378007-32BC-4175-9801-3F5A232A43F2'},
 {'_IdDocumento': '9C13BBB2-EE88-40A4-868D-CCA7829AC488'},
 {'_IdDocumento': 'A3CD0EDF-E707-4732-89F4-2925DDCDF537'},
 {'_IdDocumento': '84FEED47-E133-43FE-BF18-3E36AD16A77E'},
 {'_IdDocumento': 'C6390678-0E65-4E58-92C8-67E9A07E1103'},
 {'_IdDocumento': '2900A6D8-68B4-4432-9EEC-815DBAADE532'},
 {'_IdDocumento': '8FE310C3-F54E-4A44-903C-91AFF229CC00'},
 {'_IdDocumento': '7C95DF71-02BE-4EC1-A9FA-666381AE3DF4'},
 {'_IdDocumento': 'DB9ED836-8872-4BC3-83AC-AFAE599B31E2'},
 {'_IdDocumento': '95B08DBB-9B9F-44C4-9555-194AA1CFBCB8'},
 {'_IdDocumento': '954B1ECE-0130-4A81-9715-0DE128019602'},
 {'_IdDocumento': '94DB0028-273C-4858-AD57-B2260451AD72'},
 {'_IdDocumento': '184B5FC8-1FA4-4D74-8203-97678EDA5997'},
 {'_IdDocumento': '2BE2B83E-52C0-48DC-9192-D5511C474C13'},
 {'_IdDocumento': '094DAFE2-91F2-4E9C-BD84-02A3DE5E6B40'},
 {'_IdDocumento': 'CBBD64BC-D63D-4A22-917C-836531FF3A06'},
 {'_IdDocumento': '386B6128-F386-4627-8728-E85CDEE2B170'

In [16]:


def functionThree_ImpuestosDRCF(df):
    df_data = []
    data = functionTwo_ImpuestosDRCF(df)
    for dictionary in data:
        uuid = dictionary['_IdDocumento']
        ImpuestosDRkeys = [key for key in dictionary.keys() if ':ImpuestosDR' in key]
        
        for ImpuestosDRkey in ImpuestosDRkeys:
            ImpuestosDR = dictionary[ImpuestosDRkey]
            df_data.append({'_IdDocumento': uuid, 'ImpuestosDR': ImpuestosDR})
            new_list = df_data.copy()
    new_list = []                
    for item in df_data:
        _IdDocumento = item['_IdDocumento']
        impuestos = item.get('ImpuestosDR', {})

        for key, value in impuestos.items():
                if key.endswith(':TrasladosDR'):
                    new_item = {'_IdDocumento': _IdDocumento, 'ImpuestosDR': value}
                    new_list.append(new_item)
    
    ImpuestosDR_df = pd.DataFrame(new_list)

    return  ImpuestosDR_df

new_pagos_df = functionThree_ImpuestosDRCF(pagos_df)
new_pagos_df

KeyboardInterrupt: 

In [37]:


def functionThree_ImpuestosDRCF(df):
    df_data = []

    for dictionary in functionTwo_ImpuestosDRCF(df):
        uuid = dictionary['_IdDocumento']
        ImpuestosDRkeys = [key for key in dictionary.keys() if ':ImpuestosDR' in key]
        
        for ImpuestosDRkey in ImpuestosDRkeys:
            ImpuestosDR = dictionary[ImpuestosDRkey]
            df_data.append({'_IdDocumento': uuid, 'ImpuestosDR': ImpuestosDR})

    new_list = []   
    for item in df_data:
        _IdDocumento = item['_IdDocumento']
        impuestos = item.get('ImpuestosDR', {})
        
        for key, value in impuestos.items():
            if key.endswith(':TrasladosDR'):
                new_item = {'_IdDocumento': _IdDocumento, 'ImpuestosDR': value}
                new_list.append(new_item)

    ImpuestosDR_df = pd.DataFrame(new_list)

    return ImpuestosDR_df


In [38]:
new_pagos_df = functionThree_ImpuestosDRCF(pagos_df)
new_pagos_df

,_IdDocumento,ImpuestosDR
0,DC4FA81F-A435-46D0-96B6-5EABF361191F,{'pago20:TrasladoDR': {'0': {'_BaseDR': '40921...
1,E9A7FBCC-815B-4977-89BD-46B13744CDAF,{'pago20:TrasladoDR': {'_BaseDR': '1368460.00'...
2,40949B7B-75DF-41BC-9167-CFD6CE555525,{'pago20:TrasladoDR': {'0': {'_BaseDR': '28101...
3,37B9BECA-4836-495E-B1A5-5958C807AB2A,"{'pago20:TrasladoDR': {'_BaseDR': '44960.00', ..."
4,85AE9BA1-80BE-487B-A780-022D6E94B9FE,"{'pago20:TrasladoDR': {'_BaseDR': '144076.60',..."
...,...,...
233470,59A585B8-2621-4986-8161-147892755A49,"{'pago20:TrasladoDR': {'_BaseDR': '324.63', '_..."
233471,1A79623A-11F6-42BF-9061-C7A8103B587A,"{'pago20:TrasladoDR': {'_BaseDR': '57.2', '_Im..."
233472,EC8F81B7-E7FE-4DC6-BF45-1AD786A14540,"{'pago20:TrasladoDR': {'_BaseDR': '119.51', '_..."
233473,0539E0A8-0263-4BAC-AAA3-AF1A7BC21209,"{'pago20:TrasladoDR': {'_BaseDR': '131.53', '_..."


## Con Polars


In [35]:


def functionThree_ImpuestosDRCF(df):
    df_data = []

    data = functionTwo_ImpuestosDRCF(df)
    for dictionary in data:
        uuid = dictionary['_IdDocumento']
        ImpuestosDRkeys = [key for key in dictionary.keys() if ':ImpuestosDR' in key]
        
        for ImpuestosDRkey in ImpuestosDRkeys:
            ImpuestosDR = dictionary[ImpuestosDRkey]
            df_data.append({'_IdDocumento': uuid, 'ImpuestosDR': ImpuestosDR})

    new_list = []   
    for item in df_data:
        _IdDocumento = item['_IdDocumento']
        impuestos = item.get('ImpuestosDR', {})

        for key, value in impuestos.items():
            if key.endswith(':TrasladosDR'):
                new_item = {'_IdDocumento': _IdDocumento, 'ImpuestosDR': value}
                new_list.append(new_item)

    ImpuestosDR_df = pl.DataFrame(new_list, schema={"_IdDocumento": pl.Utf8, "ImpuestosDR": pl.Object})

    return ImpuestosDR_df


In [36]:
new_pagos_df = functionThree_ImpuestosDRCF(pagos_df)
new_pagos_df.to_pandas()


ModuleNotFoundError: No module named 'pyarrow'

In [34]:
# Tu lista de diccionarios
data = [
    {'_IdDocumento': 'uuid1', 'ImpuestosDR': {'A': 10, 'B': 20}},
    {'_IdDocumento': 'uuid2', 'ImpuestosDR': {'C': 30, 'D': 40}},
    # Agrega más diccionarios según sea necesario
]

# Convierte la lista de diccionarios en un DataFrame de Polars
# Convierte la lista de diccionarios en un DataFrame de Polars y especifica el tipo de dato
df = pl.DataFrame(data, schema={"_IdDocumento": pl.Utf8, "ImpuestosDR": pl.Object})


# Muestra el DataFrame
df

_IdDocumento,ImpuestosDR
str,object
"""uuid1""","{'A': 10, 'B': 20}"
"""uuid2""","{'C': 30, 'D': 40}"


# Extracción sobre el nuyevo dataframe

In [21]:
def functionOne_TrasladoDRCF(row):
    keys_to_find = ['_BaseDR','_ImpuestoDR','_TipoFactorDR','_TasaOCuotaDR' '_ImporteDR' ]
    keys_to_extract = keys_to_find.copy() 
    uuid = "123456789"
    try:
        uuid = row['_IdDocumento']
        traslados = row['ImpuestosDR']
        traslados_Text = str(traslados)
        cadena_json = traslados_Text.replace("'", '"')  # Reemplazar comillas simples por comillas dobles
        traslados_Json = json.loads(cadena_json)
        keys_to_find = ['_BaseDR','_ImpuestoDR','_TipoFactorDR','_TasaOCuotaDR' '_ImporteDR' ]
        keys_to_extract = keys_to_find.copy()
        result_dicts = find_dicts_with_keys(traslados_Json, keys_to_find, "_BaseDR")
        extracted_dicts = extract_keys_from_dicts(result_dicts, keys_to_extract)
        result = add_tfduuid_id(uuid, extracted_dicts)
    except Exception as e:
        print(e)
        default_values = {
   '_BaseDR': '00',
   '_ImpuestoDR': '00',
   '_TipoFactorDR': '00',
   '_TasaOCuotaDR': '00',
   '_ImporteDR': '00'
}
        result = [{'_IdDocumento': uuid, **default_values}]#'id': 00, 
    return result
#!###################################!###################################!###################################!##################################

#?##################################Función sobre los renglones del dataframe#?####################################
def functionTwo_TrasladoDRCF(df):
    results = df.apply(functionOne_TrasladoDRCF, axis=1)
    flat_list = [item for sublist in results for item in sublist]
    return flat_list
#?###################################?###################################?##################################

In [22]:
trasladoDR = functionTwo_TrasladoDRCF(new_pagos_df)
trasladoDR

[{'_IdDocumento': '8CC9CF6C-13BA-4295-A8F0-0D66752CB230',
  '_BaseDR': '1292.25',
  '_ImpuestoDR': '002',
  '_TipoFactorDR': 'Tasa'},
 {'_IdDocumento': 'A74E8953-AD73-40F5-9583-D2F88C8D6CB8',
  '_BaseDR': '123143.50',
  '_ImpuestoDR': '002',
  '_TipoFactorDR': 'Tasa'},
 {'_IdDocumento': 'B0A68754-633C-40BD-8DAD-1778B909307B',
  '_BaseDR': '207241.50',
  '_ImpuestoDR': '002',
  '_TipoFactorDR': 'Tasa'}]

In [23]:
trasladoDR = functionTwo_TrasladoDRCF(new_pagos_df)
trasladoDR_df = pd.DataFrame(trasladoDR)
trasladoDR_df.head(4)

,_IdDocumento,_BaseDR,_ImpuestoDR,_TipoFactorDR
0,8CC9CF6C-13BA-4295-A8F0-0D66752CB230,1292.25,002,Tasa
1,A74E8953-AD73-40F5-9583-D2F88C8D6CB8,123143.50,002,Tasa
2,B0A68754-633C-40BD-8DAD-1778B909307B,207241.50,002,Tasa


In [10]:
def re_build_df(df):
    
    trasladoDR_dict ={
   '_IdDocumento': "",
   '_BaseDR': '',
   '_ImpuestoDR': '',
   '_TipoFactorDR': '',
   '_TasaOCuotaDR': '',
   '_ImporteDR': ''
}
    trasladoDR_merge_df = pd.DataFrame([trasladoDR_dict])
    df = pd.merge(trasladoDR_merge_df,df, how="outer")
    df = df.drop(df.index[0])

    
    #df.rename(columns={'tfduuid': 'uuid'}, inplace=True)
    for colu in df.columns:
        if colu.startswith('_'):
            new_col = colu.replace('_','')
            df.rename(columns={colu: new_col}, inplace=True)
    numerics = ["BaseDR","ImporteDR","TasaOCuotaDR"]
    strings_col =  ["uuid","ImpuestoDR","TipoFactorDR"]
    df  = df.fillna(0)
    for colu in df.columns:
        if colu in numerics:
            df[colu] = df[colu].astype(float)
        elif colu in strings_col:
            df[colu] = df[colu].astype(str)
        # else:
        #     df[colu] = df[colu].astype(int)    
            
    new_df = df[["IdDocumento","BaseDR","ImpuestoDR","TipoFactorDR","TasaOCuotaDR","ImporteDR"]]
    return new_df
y = re_build_df(trasladoDR_df)
y.head()

,IdDocumento,BaseDR,ImpuestoDR,TipoFactorDR,TasaOCuotaDR,ImporteDR
1,5F174F17-E199-4311-8EF0-9B6C4918047C,2580.00,002,Tasa,0.0,0.0
2,03F2A6E7-2D80-4B75-BC75-9B9D2C51E10C,1112.07,002,Tasa,0.0,0.0
3,AD0AB708-C6B0-4942-8ED0-75BED028281C,25000.00,002,Tasa,0.0,0.0
4,8CC9CF6C-13BA-4295-A8F0-0D66752CB230,1292.25,002,Tasa,0.0,0.0
5,82E8C24D-9A15-46B0-9C8E-8D37B4265415,6838100.77,002,Exento,0.0,0.0


In [11]:

# Verificar si la llave "pago20:DoctoRelacionado" existe

if "pago20:Pagos" in mi_diccionario and "pago20:Pago" in mi_diccionario["pago20:Pagos"]:
    pago = mi_diccionario["pago20:Pagos"]["pago20:Pago"]
    if "pago20:DoctoRelacionado" in pago:
        docto_relacionado = pago["pago20:DoctoRelacionado"]
        print(docto_relacionado)
    else:
        print("La llave 'pago20:DoctoRelacionado' no existe. desde 20")
elif "pago10:Pagos" in mi_diccionario and "pago10:Pago" in mi_diccionario["pago10:Pagos"]:
    pago = mi_diccionario["pago10:Pagos"]["pago10:Pago"]
    if "pago10:DoctoRelacionado" in pago:
        docto_relacionado = pago["pago10:DoctoRelacionado"]
        print(docto_relacionado)
    else:
        print(pago)
else:
    print("La llave 'DoctoRelacionado' no existe.")

NameError: name 'mi_diccionario' is not defined

In [ ]:
# Verificar si la llave "pago20:DoctoRelacionado" existe
def extraccion(row):

    mi_diccionario_Text = row['ComplementoPagos']
    mi_diccionario = json.loads(mi_diccionario_Text)
    if "pago20:Pagos" in mi_diccionario and "pago20:Pago" in mi_diccionario["pago20:Pagos"]:
        pago = mi_diccionario["pago20:Pagos"]["pago20:Pago"]
        if "pago20:DoctoRelacionado" in pago:
            docto_relacionado = pago["pago20:DoctoRelacionado"]
            return docto_relacionado
        else:
            return "'pago20:DoctoRelacionado' no existe. desde 20"
    elif "pago10:Pagos" in mi_diccionario and "pago10:Pago" in mi_diccionario["pago10:Pagos"]:
        pago = mi_diccionario["pago10:Pagos"]["pago10:Pago"]
        if "pago10:DoctoRelacionado" in pago:
            docto_relacionado = pago["pago10:DoctoRelacionado"]
            return  docto_relacionado
        else:
            return "pago10:DoctoRelacionado' no existe. desde 10"
    else:
        return "LA LLAVE 'DoctoRelacionado' no existe."

In [ ]:
copia

,tfdUUID,ComplementoPagos,existe
0,004CC8AA-39BC-4AFD-846F-E6E8ECC00F33,"{\n\t""pago20:Pagos"": {\n\t\t""_Version"": ""2.0"",...",[pago20:DoctoRelacionado]
1,0145A580-6A5D-452D-A585-AE70F3E3838F,"{\n\t""pago10:Pagos"": {\n\t\t""_Version"": ""1.0"",...","[pago10:DoctoRelacionado, pago10:DoctoRelacion..."
2,0326CBFE-8C91-4150-8FB7-E879329396DF,"{\n\t""pago20:Pagos"": {\n\t\t""_Version"": ""2.0"",...",[pago20:DoctoRelacionado]
3,015DA0EB-AFC1-4946-A708-1530AB71B182,"{\n\t""pago20:Pagos"": {\n\t\t""_Version"": ""2.0"",...",[pago20:DoctoRelacionado]
4,00027530-C6AF-494F-B4B1-81E49F4450FD,"{\n\t""pago10:Pagos"": {\n\t\t""_Version"": ""1.0"",...",[pago10:DoctoRelacionado]
5,F1F1A7C2-204E-48F2-8169-9B9D2C51910A,"{\n\t""pago20:Pagos"": {\n\t\t""_Version"": ""2.0"",...",[pago20:DoctoRelacionado]
6,EAF4FBE1-ADC6-4C06-A788-A19FD184A125,"{\n\t""pago20:Pagos"": {\n\t\t""_Version"": ""2.0"",...",[pago20:DoctoRelacionado]
7,637BB1BC-0FB9-47E5-ACB9-C9E85F418207,"{\n\t""pago20:Pagos"": {\n\t\t""_Version"": ""2.0"",...",[pago20:DoctoRelacionado]
8,994A36F2-A9B3-4BA9-A6A5-ED4834C86B97,"{\n\t""pago20:Pagos"": {\n\t\t""_Version"": ""2.0"",...",[pago20:DoctoRelacionado]
9,A8E0913A-9F71-4BC8-832D-11E0F74B6303,"{\n\t""pago20:Pagos"": {\n\t\t""_Version"": ""2.0"",...",[pago20:DoctoRelacionado]


In [ ]:
new_arr = pagos_df.progress_apply(extraccion, axis=1).tolist()
dataframe = pd.DataFrame({'tipoPago': new_arr} )
copia = pd.concat([copia, dataframe], axis=1)

copia

  0%|          | 0/19 [00:00<?, ?it/s]

,tfdUUID,ComplementoPagos,existe,tipoPago
0,004CC8AA-39BC-4AFD-846F-E6E8ECC00F33,"{\n\t""pago20:Pagos"": {\n\t\t""_Version"": ""2.0"",...",[pago20:DoctoRelacionado],"{'_EquivalenciaDR': '1', '_Folio': '69919732',..."
1,0145A580-6A5D-452D-A585-AE70F3E3838F,"{\n\t""pago10:Pagos"": {\n\t\t""_Version"": ""1.0"",...","[pago10:DoctoRelacionado, pago10:DoctoRelacion...",pago10:DoctoRelacionado' no existe. desde 10
2,0326CBFE-8C91-4150-8FB7-E879329396DF,"{\n\t""pago20:Pagos"": {\n\t\t""_Version"": ""2.0"",...",[pago20:DoctoRelacionado],"{'_EquivalenciaDR': '1', '_IdDocumento': '03F2..."
3,015DA0EB-AFC1-4946-A708-1530AB71B182,"{\n\t""pago20:Pagos"": {\n\t\t""_Version"": ""2.0"",...",[pago20:DoctoRelacionado],"{'_EquivalenciaDR': '1', '_Folio': '40', '_IdD..."
4,00027530-C6AF-494F-B4B1-81E49F4450FD,"{\n\t""pago10:Pagos"": {\n\t\t""_Version"": ""1.0"",...",[pago10:DoctoRelacionado],"{'0': {'_Folio': 'D0000160439', '_IdDocumento'..."
5,F1F1A7C2-204E-48F2-8169-9B9D2C51910A,"{\n\t""pago20:Pagos"": {\n\t\t""_Version"": ""2.0"",...",[pago20:DoctoRelacionado],"{'_EquivalenciaDR': '1', '_Folio': '59218059',..."
6,EAF4FBE1-ADC6-4C06-A788-A19FD184A125,"{\n\t""pago20:Pagos"": {\n\t\t""_Version"": ""2.0"",...",[pago20:DoctoRelacionado],"{'0': {'_EquivalenciaDR': '1', '_Folio': '8722..."
7,637BB1BC-0FB9-47E5-ACB9-C9E85F418207,"{\n\t""pago20:Pagos"": {\n\t\t""_Version"": ""2.0"",...",[pago20:DoctoRelacionado],"{'0': {'_EquivalenciaDR': '1', '_Folio': '8779..."
8,994A36F2-A9B3-4BA9-A6A5-ED4834C86B97,"{\n\t""pago20:Pagos"": {\n\t\t""_Version"": ""2.0"",...",[pago20:DoctoRelacionado],"{'0': {'_EquivalenciaDR': '1', '_Folio': '8696..."
9,A8E0913A-9F71-4BC8-832D-11E0F74B6303,"{\n\t""pago20:Pagos"": {\n\t\t""_Version"": ""2.0"",...",[pago20:DoctoRelacionado],"{'0': {'_EquivalenciaDR': '1', '_Folio': '8601..."


In [ ]:
copia.to_csv('prueba.csv')

In [ ]:
matriz_numpy = np.array(new_arr)
matriz_numpy[0]

{'_EquivalenciaDR': '1',
 '_Folio': '69919732',
 '_IdDocumento': '5F174F17-E199-4311-8EF0-9B6C4918047C',
 '_ImpPagado': '2992.80',
 '_ImpSaldoAnt': '2992.80',
 '_ImpSaldoInsoluto': '0.00',
 '_MonedaDR': 'MXN',
 '_NumParcialidad': '1',
 '_ObjetoImpDR': '02',
 '_Serie': 'CFDI',
 'pago20:ImpuestosDR': {'pago20:TrasladosDR': {'pago20:TrasladoDR': {'_BaseDR': '2580.00',
    '_ImporteDR': '412.80',
    '_ImpuestoDR': '002',
    '_TasaOCuotaDR': '0.160000',
    '_TipoFactorDR': 'Tasa'}}}}

In [ ]:
patron_pagos = re.compile(r'^pago\d{2}:Pagos$')
patron_docto_relacionado = re.compile(r'^pago\d{2}:DoctoRelacionado*')

pago_key = next((key for key in mi_diccionario.keys() if patron_pagos.match(key)), None)

if pago_key is not None and "pago10:Pago" in mi_diccionario[pago_key]:
    pago = mi_diccionario[pago_key]["pago10:Pago"]
    docto_relacionado_key = next((key for key in pago.keys() if patron_docto_relacionado.match(key)), None)
    
    if docto_relacionado_key is not None:
        docto_relacionado = pago[docto_relacionado_key]
        print(docto_relacionado)
    else:
        print("No se encontró una clave que coincida con el patrón en 'pago20:Pago'.")
else:
    print("No se encontró una clave que coincida con el patrón en el diccionario.")

{'_Folio': 'D0000124685', '_IdDocumento': 'FE01E392-9F66-4CEC-9842-502CB801BFB0', '_ImpPagado': '25564.00', '_ImpSaldoAnt': '25564.00', '_ImpSaldoInsoluto': '0', '_MetodoDePagoDR': 'PPD', '_MonedaDR': 'MXN', '_NumParcialidad': '1', '_Serie': 'CFDI'}


In [ ]:
diccionario = {'_EquivalenciaDR': '1',
 '_Folio': '69919732',
 '_IdDocumento': '5F174F17-E199-4311-8EF0-9B6C4918047C',
 '_ImpPagado': '2992.80',
 '_ImpSaldoAnt': '2992.80',
 '_ImpSaldoInsoluto': '0.00',
 '_MonedaDR': 'MXN',
 '_NumParcialidad': '1',
 '_ObjetoImpDR': '02',
 '_Serie': 'CFDI',
 'pago20:ImpuestosDR': {'pago20:TrasladosDR': {'pago20:TrasladoDR': {'_BaseDR': '2580.00',
    '_ImporteDR': '412.80',
    '_ImpuestoDR': '002',
    '_TasaOCuotaDR': '0.160000',
    '_TipoFactorDR': 'Tasa'}}}}

## PUEBA NUEVA ESTRUCTURA

In [ ]:
def explore_nested_structure(data, indent=''):
    if isinstance(data, dict):
        for key, value in data.items():
            print(f"{indent}{key}:")
            explore_nested_structure(value, indent + '  ')
    elif isinstance(data, list):
        for index, item in enumerate(data):
            print(f"{indent}[{index}]:")
            explore_nested_structure(item, indent + '  ')
    else:
        print(f"{indent}{data}")



# Llamada a la función para explorar las estructuras anidadas
explore_nested_structure(mi_diccionario)

pago10:Pagos:
  _Version:
    1.0
  pago10:Pago:
    _FechaPago:
      2022-02-10T16:00:00
    _FormaDePagoP:
      03
    _MonedaP:
      MXN
    _Monto:
      243757.00
    pago10:DoctoRelacionado.0:
      _Folio:
        D0000124685
      _IdDocumento:
        FE01E392-9F66-4CEC-9842-502CB801BFB0
      _ImpPagado:
        25564.00
      _ImpSaldoAnt:
        25564.00
      _ImpSaldoInsoluto:
        0
      _MetodoDePagoDR:
        PPD
      _MonedaDR:
        MXN
      _NumParcialidad:
        1
      _Serie:
        CFDI
    pago10:DoctoRelacionado.1:
      _Folio:
        D0000124685
      _IdDocumento:
        69F324ED-0705-4AB2-88BF-7A5BC302312F
      _ImpPagado:
        22619.00
      _ImpSaldoAnt:
        22619.00
      _ImpSaldoInsoluto:
        0
      _MetodoDePagoDR:
        PPD
      _MonedaDR:
        MXN
      _NumParcialidad:
        1
      _Serie:
        CFDI
    pago10:DoctoRelacionado.10:
      _Folio:
        D0000124685
      _IdDocumento:
        7B692253-DEDF

In [ ]:
def explore_nested_structure(data, level_lists, level=0):
    if level >= len(level_lists):
        level_lists.append([])

    if isinstance(data, dict):
        level_list = level_lists[level]
        for key in data.keys():
            level_list.append(key)
            explore_nested_structure(data[key], level_lists, level + 1)

# Tu diccionario
tu_diccionario = {
    # ... (contenido de tu diccionario)
}

# Listas para almacenar las llaves por nivel
primer_nivel = []
segundo_nivel = []
tercer_nivel = []
cuarto_nivel = []
quinto_nivel = []
sexto_nivel = []
septimo_nivel = []
octavo_nivel = []
noveno_nivel = []
decimo_nivel = []

# Lista de todas las listas de niveles
level_lists = [primer_nivel, segundo_nivel, tercer_nivel, cuarto_nivel, quinto_nivel,
               sexto_nivel, septimo_nivel, octavo_nivel, noveno_nivel, decimo_nivel]

# Llamada a la función para explorar las estructuras anidadas y guardar las llaves por nivel
explore_nested_structure(mi_diccionario, level_lists)

# Imprimir las listas de llaves por nivel
for i, nivel in enumerate(level_lists, start=1):
    print(f"Nivel {i}:", nivel)

Nivel 1: ['pago10:Pagos', 'tfd:TimbreFiscalDigital']
Nivel 2: ['_Version', 'pago10:Pago', '_FechaTimbrado', '_NoCertificadoSAT', '_RfcProvCertif', '_SelloCFD', '_SelloSAT', '_UUID', '_Version', '_xmlns:tfd', '_xsi:schemaLocation']
Nivel 3: ['_FechaPago', '_FormaDePagoP', '_MonedaP', '_Monto', 'pago10:DoctoRelacionado.0', 'pago10:DoctoRelacionado.1', 'pago10:DoctoRelacionado.10', 'pago10:DoctoRelacionado.11', 'pago10:DoctoRelacionado.2', 'pago10:DoctoRelacionado.3', 'pago10:DoctoRelacionado.4', 'pago10:DoctoRelacionado.5', 'pago10:DoctoRelacionado.6', 'pago10:DoctoRelacionado.7', 'pago10:DoctoRelacionado.8', 'pago10:DoctoRelacionado.9']
Nivel 4: ['_Folio', '_IdDocumento', '_ImpPagado', '_ImpSaldoAnt', '_ImpSaldoInsoluto', '_MetodoDePagoDR', '_MonedaDR', '_NumParcialidad', '_Serie', '_Folio', '_IdDocumento', '_ImpPagado', '_ImpSaldoAnt', '_ImpSaldoInsoluto', '_MetodoDePagoDR', '_MonedaDR', '_NumParcialidad', '_Serie', '_Folio', '_IdDocumento', '_ImpPagado', '_ImpSaldoAnt', '_ImpSaldoInso

In [ ]:
def find_keys_upwards(data, target_key, path=[]):
    keys = []
    if isinstance(data, dict):
        for key, value in data.items():
            new_path = path + [key]
            if key == target_key:
                keys.append(new_path)
            keys.extend(find_keys_upwards(value, target_key, new_path))
    elif isinstance(data, list):
        for idx, item in enumerate(data):
            new_path = path + [str(idx)]
            keys.extend(find_keys_upwards(item, target_key, new_path))
    return keys

# Tu diccionario


# Llave objetivo
target_key = "pago20:TrasladosDR"

# Encontrar las llaves anidadas hacia arriba de la llave objetivo
keys_upwards = find_keys_upwards(mi_diccionario, target_key)

# Imprimir la lista de llaves anidadas hacia arriba
for keys in keys_upwards:
    print("Llaves anidadas hacia arriba de", target_key, ":", keys)

In [ ]:
def find_key_and_values(data, target_key):
    result = []
    
    if isinstance(data, dict):
        if target_key in data:
            result.append(data[target_key])
        
        for value in data.values():
            result.extend(find_key_and_values(value, target_key))
    elif isinstance(data, list):
        for item in data:
            result.extend(find_key_and_values(item, target_key))
    
    return result

# Tu diccionario


# Llave objetivo
target_key = "pago20:TrasladosDR"

# Buscar la llave y obtener sus valores
valores_traslados = find_key_and_values(mi_diccionario, target_key)

# Imprimir los valores encontrados
for valores in valores_traslados:
    print(valores)

In [ ]:
def find_key_and_value(data, target_keys):
    result = []
    
    if isinstance(data, dict):
        for key, value in data.items():
            if key in target_keys:
                result.append({key: value})
            
            result.extend(find_key_and_value(value, target_keys))
    elif isinstance(data, list):
        for item in data:
            result.extend(find_key_and_value(item, target_keys))
    
    return result

# Tu diccionario


# Llaves objetivo
target_keys = ["pago20:TrasladosDR", "pago10:TrasladosDR"]

# Buscar las llaves y obtener los diccionarios o valores correspondientes
resultados = find_key_and_value(mi_diccionario, target_keys)

# Imprimir los resultados encontrados
for resultado in resultados:
    print(resultado)

In [ ]:
def find_key_and_value(data, target_keys):
    result = []
    
    if isinstance(data, dict):
        for key, value in data.items():
            if key in target_keys:
                result.append({key: value})
            
            result.extend(find_key_and_value(value, target_keys))
    elif isinstance(data, list):
        for item in data:
            result.extend(find_key_and_value(item, target_keys))

    
    return result

target_keys = ["pago20:TrasladosDR", "pago10:TrasladosDR"]

    # Buscar las llaves y obtener los diccionarios o valores correspondientes
resultados = find_key_and_value(mi_diccionario, target_keys)
resultados

[]

In [ ]:

target_keys = ["pago20:TrasladoDR", "pago10:TrasladoDR"]

def other(row):
    tfduuid = row['tfdUUID']
    complementoPagos_Text = row['ComplementoPagos']
    complementoPagos_Json = json.loads(complementoPagos_Text)
    resultados = find_key_and_value(complementoPagos_Json , target_keys)
    if resultados != []:
        for resultado in resultados:
                r = list(resultado.keys())
                for llave in r:
                    to_return = resultado[llave]
                    to_return['tfdUUID'] = tfduuid
                    return to_return
    else:
        return {}

            


In [ ]:
def other_other(df):
    return df.progress_apply(other, axis=1).tolist()

In [ ]:
other_other(pagos_df)
new = [jsn for jsn in other_other(pagos_df) if jsn !={}]
new

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

[{'_BaseDR': '2580.00',
  '_ImporteDR': '412.80',
  '_ImpuestoDR': '002',
  '_TasaOCuotaDR': '0.160000',
  '_TipoFactorDR': 'Tasa',
  'tfdUUID': '004CC8AA-39BC-4AFD-846F-E6E8ECC00F33'},
 {'_BaseDR': '1112.07',
  '_ImporteDR': '177.93',
  '_ImpuestoDR': '002',
  '_TasaOCuotaDR': '0.160000',
  '_TipoFactorDR': 'Tasa',
  'tfdUUID': '0326CBFE-8C91-4150-8FB7-E879329396DF'},
 {'_BaseDR': '25000.00',
  '_ImporteDR': '4000.00',
  '_ImpuestoDR': '002',
  '_TasaOCuotaDR': '0.160000',
  '_TipoFactorDR': 'Tasa',
  'tfdUUID': '015DA0EB-AFC1-4946-A708-1530AB71B182'},
 {'_BaseDR': '1292.25',
  '_ImporteDR': '206.760000',
  '_ImpuestoDR': '002',
  '_TasaOCuotaDR': '0.160000',
  '_TipoFactorDR': 'Tasa',
  'tfdUUID': 'F1F1A7C2-204E-48F2-8169-9B9D2C51910A'},
 {'_BaseDR': '6838100.77',
  '_ImpuestoDR': '002',
  '_TipoFactorDR': 'Exento',
  'tfdUUID': 'EAF4FBE1-ADC6-4C06-A788-A19FD184A125'},
 {'_BaseDR': '13347469.06',
  '_ImpuestoDR': '002',
  '_TipoFactorDR': 'Exento',
  'tfdUUID': '637BB1BC-0FB9-47E5-AC

## Creación de los dataframes para volcar a las tablas

### Dataframe para pago


In [ ]:
pago_dict ={
   'tfdUUID': "",
   "_FechaPago": "",
   "_FormaDePagoP": "",
   "_MonedaP": "",
   "_TipoCambioP": "",
   "_Monto": "",
   "_NumOperacion": "",
   "_RfcEmisorCtaOrd": "",
   "_NomBancoOrdExt": "",
   "_CtaOrdenante": "",
   "_RfcEmisorCtaBen": "",
   "_CtaBeneficiario": "",
   "_TipoCadPago": "",
   "_CertPago": "",
   "_CadPago": "",
   "_SelloPago": ""
}


pago_df = pd.DataFrame([pago_dict])
pago_df

,tfdUUID,_FechaPago,_FormaDePagoP,_MonedaP,_TipoCambioP,_Monto,_NumOperacion,_RfcEmisorCtaOrd,_NomBancoOrdExt,_CtaOrdenante,_RfcEmisorCtaBen,_CtaBeneficiario,_TipoCadPago,_CertPago,_CadPago,_SelloPago
0,,,,,,,,,,,,,,,,


## Dataframe para Documento Relacionado

In [ ]:
docto_relacionado_dict = {
    "tfdUUID": "",
    "id": "",
    "_IdDocumento": "",
    "_Serie": "",
    "_Folio": "",
    "_MonedaDR": "",
    "_EquivalenciaDR": "",
    "_NumParcialidad": "",
    "_ImpSaldoAnt": "",
    "_ImpPagado": "",
    "_ImpSaldoInsoluto": "",
    "_ObjetoImpDR": "",
    
}

docto_relacionado_df = pd.DataFrame([docto_relacionado_dict])
docto_relacionado_df

,tfdUUID,id,_IdDocumento,_Serie,_Folio,_MonedaDR,_EquivalenciaDR,_NumParcialidad,_ImpSaldoAnt,_ImpPagado,_ImpSaldoInsoluto,_ObjetoImpDR
0,,,,,,,,,,,,


## Navegación por medio de expresiones regulares

In [ ]:
pagos_regex1l = r'pago\d{1,3}:Pagos'
pagoTotales_regex2l = r'pago\d{1,3}:Totales'
pago_regex2l = r'pago\d{1,3}:Pago'
doctoRel_regex3l = r'pago\d{1,2}:DoctoRelacionado(?:\.\d{1,2})?'


## Primer nivel


In [ ]:
complemento_Json = json.loads(pagos_df['ComplementoPagos'][12])
complemento_Text = str(complemento_Json)

#Todas las coincidencias en el texto plano del primer nivel
searchPagos = re.finditer(pagos_regex1l, complemento_Text)
res_pagos = []
for pagos in searchPagos:
    valor = pagos.group()      # Obtener el valor de la coincidencia
    res_pagos.append(valor)
    
pagos_Json1l = complemento_Json[res_pagos[0]]
pagos_Text1l = str(pagos_Json1l)



## Segundo Nivel

In [ ]:
#Todas las coincidencias en el texto plano del segundo nivel

searchPago = re.finditer(pago_regex2l, pagos_Text1l)
res_pago = []
for pago in searchPago:
    valor = pago.group()      # Obtener el valor de la coincidencia
    res_pago.append(valor)

pago_Json2l = pagos_Json1l[res_pago[0]]
pago_Text2l = str(pago_Json2l)

res_pago

['pago20:Pago']

## Tercer Nivel

In [ ]:
#Todas las coincidencias en el texto plano del tercer nivel
searchDocto = re.findall(doctoRel_regex3l, pago_Text2l)
docto_Json3l = pago_Json2l[searchDocto[0]]
docto_Json3l.items()
docto_Json3l.keys()

dict_keys(['_EquivalenciaDR', '_Folio', '_IdDocumento', '_ImpPagado', '_ImpSaldoAnt', '_ImpSaldoInsoluto', '_MonedaDR', '_NumParcialidad', '_ObjetoImpDR', '_Serie', 'pago20:ImpuestosDR'])

In [ ]:
for x,y in enumerate(docto_Json3l.keys()):
    print(x,y)

0 _EquivalenciaDR
1 _Folio
2 _IdDocumento
3 _ImpPagado
4 _ImpSaldoAnt
5 _ImpSaldoInsoluto
6 _MonedaDR
7 _NumParcialidad
8 _ObjetoImpDR
9 _Serie
10 pago20:ImpuestosDR


## Cuarto nivel. Iterar sobre las llaves 

In [ ]:
def iter_keys(json, tfUUID):
    y = docto_relacionado_df.copy()
    concated_df = docto_relacionado_df.copy()
    for j,i in enumerate(json.keys()):
        to_merge = pd.DataFrame([json[i]])
        df_toConcat = pd.merge(y, to_merge, how = 'outer')
        df_toConcat['tfdUUID'] = tfUUID
        df_toConcat['id'] = j+1
        concated_df = pd.concat([concated_df, df_toConcat], ignore_index=True)
        concated_df = concated_df[y.columns]
        
        concated_df = concated_df[concated_df['_IdDocumento'] != '']
        
    return concated_df
test = iter_keys(docto_Json3l, "skhry")
test

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

## Implementacion de la lógica para iterar sobre todos los pagos y extraer totales

In [ ]:
def iter_keys(json, tfUUID):
    y = docto_relacionado_df.copy()
    concated_df = docto_relacionado_df.copy()
    for j,i in enumerate(json.keys()):
        to_merge = pd.DataFrame([json[i]])
        df_toConcat = pd.merge(y, to_merge, how = 'outer')
        df_toConcat['tfdUUID'] = tfUUID
        df_toConcat['id'] = j+1
        concated_df = pd.concat([concated_df, df_toConcat], ignore_index=True)
        concated_df = concated_df[y.columns]
        
        concated_df = concated_df[concated_df['_IdDocumento'] != '']
        
    return concated_df

def functionOne_Documento (row):
    y = docto_relacionado_df.copy()
    
    
    pagos_regex1l = r'pago\d{1,3}:Pagos'
    pago_regex2l = r'pago\d{1,3}:Pago'
    doctoRel_regex3l = r'pago\d{1,2}:DoctoRelacionado(?:\.\d{1,2})?'
    tfduuid = row['tfdUUID']
    complementoPagos_Text = row['ComplementoPagos']
    complementoPagos_Json = json.loads(complementoPagos_Text)
   ###########################################################################
    #Primer nivel
   ###########################################################################
    #Todas las coincidencias en el texto plano del primer nivel
    searchPagos = re.finditer(pagos_regex1l, complementoPagos_Text)
    res_pagos = []
    for pagos in searchPagos:
        valor = pagos.group()      # Obtener el valor de la coincidencia
        res_pagos.append(valor)
        
    pagos_Json1l = complementoPagos_Json[valor]
    pagos_Text1l = str(pagos_Json1l)
    
    ###################################################################################
    #Todas las coincidencias en el texto plano del segundo nivel
    searchPago = re.findall(pago_regex2l, pagos_Text1l)
    if searchPago == []:
        pass
    else:
        pago_Json2l = pagos_Json1l[searchPago[0]]
        pago_Text2l = str(pago_Json2l)
    #Todas las coincidencias en el texto plano del tercer nivel
    searchDocto = re.findall(doctoRel_regex3l, pago_Text2l)
    concated_df = docto_relacionado_df.copy()
    
    
    for ind, docto in enumerate(searchDocto):
        docto_Json3l = pago_Json2l[docto]
        if "0" in docto_Json3l:
            test = iter_keys(docto_Json3l, tfduuid)
            concated_df = pd.concat([concated_df, test], ignore_index=True)
            concated_df = concated_df[y.columns]
            concated_df = concated_df[concated_df['_IdDocumento'] != '']
        else:
            to_merge = pd.DataFrame([docto_Json3l])
            df_toConcat = pd.merge(y, to_merge, how = 'outer')
            df_toConcat['tfdUUID'] = tfduuid
            df_toConcat['id'] = ind+1
            concated_df = pd.concat([concated_df, df_toConcat], ignore_index=True)
            concated_df = concated_df[y.columns]
            
            concated_df = concated_df[concated_df['_IdDocumento'] != '']
        return concated_df




In [ ]:
def functionTwo_documento(df):
    return df.progress_apply(functionOne_Documento, axis=1).tolist()

## Ejecutar la función

In [ ]:
documentos_final_df = docto_relacionado_df.copy()
for ele in tqdm(functionTwo_documento(pagos_df)):
    documentos_final_df = pd.concat([documentos_final_df, ele], ignore_index=True)
documentos_final_df = documentos_final_df.drop([0], axis=0)
#documentos_final_df
documentos_final_df.to_csv('984registros.csv', index= False)

  0%|          | 0/984 [00:00<?, ?it/s]

  0%|          | 0/984 [00:00<?, ?it/s]

In [ ]:
documentos_final_df.shape

(1739, 12)

# Pruebas

In [ ]:
expe_dic = {
    "tfdUUID": "error",
    "id": "error",
    "_IdDocumento": "error",
    "_Serie": "error",
    "_Folio": "error",
    "_MonedaDR": "error",
    "_EquivalenciaDR": "error",
    "_NumParcialidad": "error",
    "_ImpSaldoAnt": "error",
    "_ImpPagado": "error",
    "_ImpSaldoInsoluto": "error",
    "_ObjetoImpDR": "error",
    
}

expe_df = pd.DataFrame([expe_dic])
expe_df

: 

In [ ]:
def functionOne_Documento (row):
    y = docto_relacionado_df.copy()
    
    
    pagos_regex1l = r'pago\d{1,3}:Pagos'
    pago_regex2l = r'pago\d{1,3}:Pago'
    doctoRel_regex3l = r'pago\d{1,2}:DoctoRelacionado(?:\.\d{1,2})?'
    tfduuid = row['tfdUUID']
    complementoPagos_Text = row['ComplementoPagos']
    complementoPagos_Json = json.loads(complementoPagos_Text)
   ###########################################################################
    #Primer nivel
   ###########################################################################
    #Todas las coincidencias en el texto plano del primer nivel
    searchPagos = re.finditer(pagos_regex1l, complementoPagos_Text)
    res_pagos = []
    for pagos in searchPagos:
        valor = pagos.group()      # Obtener el valor de la coincidencia
        res_pagos.append(valor)
        
    pagos_Json1l = complementoPagos_Json[valor]
    pagos_Text1l = str(pagos_Json1l)
    
    ###################################################################################
    #Todas las coincidencias en el texto plano del segundo nivel
    searchPago = re.findall(pago_regex2l, pagos_Text1l)
    if searchPago == []:
        pass
    else:
        pago_Json2l = pagos_Json1l[searchPago[0]]
        pago_Text2l = str(pago_Json2l)
    #Todas las coincidencias en el texto plano del tercer nivel
    searchDocto = re.findall(doctoRel_regex3l, pago_Text2l)
    concated_df = docto_relacionado_df.copy()
    
    
    for ind, docto in enumerate(searchDocto):
        docto_Json3l = pago_Json2l[docto]
        if "0" in docto_Json3l:
            test = iter_keys(docto_Json3l, tfduuid)
            concated_df = pd.concat([concated_df, test], ignore_index=True)
            concated_df = concated_df[y.columns]
            concated_df = concated_df[concated_df['_IdDocumento'] != '']
        else:
            to_merge = pd.DataFrame([docto_Json3l])
            df_toConcat = pd.merge(y, to_merge, how = 'outer')
            df_toConcat['tfdUUID'] = tfduuid
            df_toConcat['id'] = ind+1
            concated_df = pd.concat([concated_df, df_toConcat], ignore_index=True)
            concated_df = concated_df[y.columns]
            
            concated_df = concated_df[concated_df['_IdDocumento'] != '']
        return concated_df
             
        
        #return aver
        # to_merge = pd.DataFrame([docto_Json3l])
        # df_toConcat = pd.merge(y, to_merge, how = 'outer')
        # df_toConcat['tfdUUID'] = tfduuid
        # df_toConcat['id'] = ind+1
        # concated_df = pd.concat([concated_df, df_toConcat], ignore_index=True)
        # concated_df = concated_df[y.columns]
        
        #concated_df = concated_df[concated_df['_IdDocumento'] != '']
    #return docto_Json3l.keys()
    
        #df_toConcat = pd.DataFrame(merged_df)
        #df_test1 = pd.concat([df_test1, df_toConcat], ignore_index=True)
    
    #df_cleaned = df_test1[df_test1['tfdUUID'] != '']
    #df_cleaned = df_cleaned.drop(df_cleaned.columns[-79:], axis=1)
    #df_cleaned.to_csv('test_900.csv', index= False)
        


: 

In [ ]:
def functionTwo_documento(df):
    return df.progress_apply(functionOne_Documento, axis=1).tolist()
l = functionTwo_documento(pagos_df)

: 

In [ ]:
l[12]

: 

In [ ]:
docto_relacionado_df

: 

In [ ]:
documentos_final_df = docto_relacionado_df.copy()
for ele in tqdm(functionTwo_documento(pagos_df)):
    documentos_final_df = pd.concat([documentos_final_df, ele], ignore_index=True)
documentos_final_df = documentos_final_df.drop([0], axis=0)
#documentos_final_df
documentos_final_df.to_csv('test_test_fuego.csv', index= False)

: 

In [ ]:
l[11]

: 

In [ ]:
df_toConcat = docto_relacionado_df.copy()
for ele in tqdm(functionTwo_pago(pagos_df)):
    df_toConcat = pd.concat([df_toConcat, ele], ignore_index=True)

: 

In [ ]:
df_toConcat


: 

In [ ]:
#Todas las coincidencias en el texto plano del tercer nivel
searchDocto = re.finditer(doctoRel_regex3l, pago_Text2l)
res_docto = []
for docto in searchDocto:
    valor = docto.group()      # Obtener el valor de la coincidencia
    res_docto.append(valor)
    
docto_Json3l = pago_Json2l[res_docto[0]]
docto_Json3l.items()

res_docto 

: 

In [ ]:
def functionTwo_pago(df):
    return df.progress_apply(functionOne_Pago, axis=1).tolist()
functionTwo_pago(pagos_df)

: 

In [ ]:
df_test1 = pago_df.copy()
y = pago_df.copy()
#merged_df = pd.merge(y, x, how = 'outer')
#merged_df
for ele in tqdm(functionTwo_pago(pagos_df)):
    to_merge = pd.DataFrame([ele])
    merged_df = pd.merge(y, to_merge, how = 'outer')
    df_toConcat = pd.DataFrame(merged_df)
    df_test1 = pd.concat([df_test1, df_toConcat], ignore_index=True)
    
df_cleaned = df_test1[df_test1['tfdUUID'] != '']
df_cleaned = df_cleaned.drop(df_cleaned.columns[-79:], axis=1)
df_cleaned.to_csv('test_900.csv', index= False)

: 

In [ ]:
df_cleaned

: 

: 